In [1]:
# required packages: countdown-timer pyserial numpy av

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import subprocess
import datetime
import multiprocessing as mp
import sys, time
#sys.path.append('simple_pyspin/')
from multicamera_acquisition.simple_pyspin import Camera
import PySpin
from countdown import countdown
import av

In [4]:
#from llpyspin import secondary, primary
import numpy as np
import matplotlib.pyplot as plt
import os, time, serial, glob, struct

def packIntAsLong(value):
    """Packs a python 4 byte integer to an arduino long"""
    return struct.pack('i', value)

    
def get_timing_params(duration, framerate, exposure_time, buffer=50):
    
    inv_framerate = int(1e6/framerate)
    num_cycles = int(duration * framerate / 2)
    
    params = (
        num_cycles,
        exposure_time,
        inv_framerate,
    )
    
    return params
    

In [5]:
#num_cycles = number of on off cycles
#exposure_time = duration of exposure
#inv_framerate,1
#phase_shift,
#UV_duration,
#UV_delay

In [6]:
def count_frames(file_name):
    with av.open(file_name, 'r') as reader:
        return reader.streams.video[0].frames


                
def write_frames(filename, frames, 
                 threads=6, fps=30, crf=10,
                 pixel_format='gray8', codec='ffv1',
                 pipe=None, slices=24, slicecrc=1):
    
    frame_size = '{0:d}x{1:d}'.format(frames.shape[2], frames.shape[1])
    command = ['ffmpeg',
               '-y',
               '-loglevel', 'fatal',
               '-framerate', str(fps),
               '-f', 'rawvideo',
               '-s', frame_size,
               '-pix_fmt', pixel_format,
               '-i', '-',
               '-an',
               '-crf',str(crf),
               '-vcodec', codec,
               '-preset', 'ultrafast',
               '-threads', str(threads),
               '-slices', str(slices),
               '-slicecrc', str(slicecrc),
               '-r', str(fps),
               filename]

    if not pipe: pipe = subprocess.Popen(command, stdin=subprocess.PIPE, stderr=subprocess.PIPE)
    dtype = np.uint16 if pixel_format.startswith('gray16') else np.uint8
    for i in range(frames.shape[0]): pipe.stdin.write(frames[i,:,:].astype(dtype).tobytes())
    return pipe


In [7]:
def get_camera(serial_number=None, exposure_time=2000, gain=15):
    
    cam = Camera(index=str(serial_number))
    cam.init()
    
    cam.GainAuto = 'Off'
    cam.Gain = gain
    cam.ExposureAuto = 'Off'
    cam.ExposureTime = exposure_time
    cam.AcquisitionMode = 'Continuous'
    
    cam.AcquisitionFrameRateEnable = True
    max_fps = cam.get_info('AcquisitionFrameRate')['max']
    cam.AcquisitionFrameRate = max_fps
    
    cam.TriggerMode = 'Off'
    cam.TriggerSource = 'Line3'
    cam.TriggerOverlap = 'ReadOut'
    cam.TriggerSelector = 'FrameStart'
    cam.TriggerActivation = 'RisingEdge'
    cam.TriggerMode = 'On'
    
    return cam

In [8]:
class AcquisitionLoop(mp.Process):
    
    def __init__(self, write_queue, **camera_params):
        super().__init__()

        self.ready = mp.Event()
        self.primed = mp.Event()
        self.stopped = mp.Event()
        self.write_queue = write_queue
        self.camera_params = camera_params

    def stop(self):
        self.stopped.set()
        
    def prime(self):
        self.ready.clear()
        self.primed.set()
        
    def run(self):
        try:
            cam = get_camera(**self.camera_params) 
        except:
            print(f'Failed to get camera {self.camera_params["serial_number"]}')
            
        self.ready.set()
        self.primed.wait()
        
        cam.start()
        self.ready.set()
        
        while not self.stopped.is_set():
            try: 
                data = cam.get_array(timeout=1000, get_timestamp=True)
                self.write_queue.put(data)
            except PySpin.SpinnakerException: 
                pass 

        self.write_queue.put(tuple())
        if cam is not None: cam.close()

In [9]:
class Writer(mp.Process):
    
    def __init__(self, queue, file_name, **ffmpeg_options):
        super().__init__()
        self.pipe = None
        self.queue = queue
        self.file_name = file_name
        self.ffmpeg_options = ffmpeg_options

    def run(self):  
        while True:
            data = self.queue.get()
            img, camera_timestamp = data
            #print(data)
            if len(data)==0: 
                break
            else: 
                self.append(img)
        self.close()
            
    def append(self, data):
        self.pipe = write_frames(
            self.file_name, data[None], 
            pipe=self.pipe, **self.ffmpeg_options)

    def close(self):
        if self.pipe is not None:
            self.pipe.stdin.close()

In [10]:
prefix = '../../data/test'

duration = 3
framerate = 150
exposure_time = 2000

params = get_timing_params(duration, framerate, exposure_time)
print(params)

(225, 2000, 6666)


In [11]:
serial_nums = {
#   'top':    22181547,
   'side1':  22181612,
}


In [12]:
# initialize cameras
writers = []
acquisition_loops = []

for k,sn in serial_nums.items():

    write_queue = mp.Queue()
    
    writer = Writer(
        write_queue,
        f'{prefix}.{k}.avi',
        fps=framerate)
    
    acquisition_loop = AcquisitionLoop(
        write_queue, 
        serial_number=sn, 
        exposure_time=exposure_time,
        gain=15)
    
    writer.start()
    writers.append(writer)
    acquisition_loop.start()
    acquisition_loop.ready.wait()
    acquisition_loops.append(acquisition_loop)
    print(f'Initialized {k}')

Initialized side1


Process Writer-1:
Traceback (most recent call last):
  File "/home/dattalab/anaconda3/envs/flir_acq/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/tmp/ipykernel_10797/469361115.py", line 13, in run
    img, camera_timestamp = data
ValueError: not enough values to unpack (expected 2, got 0)


In [13]:
# prepare acquisition loops
for acquisition_loop in acquisition_loops:
    acquisition_loop.prime()
    acquisition_loop.ready.wait()

In [14]:
#`sudo chmod a+rw /dev/ttyACM0`

In [15]:
port = glob.glob('/dev/ttyACM*')[0]
arduino = serial.Serial(port=port)

In [16]:
params

(225, 2000, 6666)

In [17]:
[struct.pack('i', value) for value in params]

[b'\xe1\x00\x00\x00', b'\xd0\x07\x00\x00', b'\n\x1a\x00\x00']

In [19]:
msg = b''.join(map(packIntAsLong,params))
arduino.write(msg)

12

In [20]:
msg = b''.join(map(packIntAsLong,params))
arduino.write(msg)

12

In [21]:
start_confirmation = arduino.readline().decode('utf-8').strip('\r\n')
print('Arduino confirmation:',start_confirmation)

if start_confirmation == "Start":
    print('\nRecorded:')
    countmapvideo_file_namedown(mins=0, secs=duration)
    end_confirmation = arduino.readline().decode('utf-8').strip('\r\n')
    print('Arduino confirmation:',end_confirmation)
else:
    raise ValueError('Start confirmation not available')

Arduino confirmation: Start

Recorded:
Arduino confirmation: Finished


In [22]:
# end acquisition loops
for acquisition_loop in acquisition_loops:
    acquisition_loop.stop()
    acquisition_loop.join()

In [23]:
for writer in writers: writer.join()

In [24]:
try:
    print('Frame counts', [count_frames(f'{prefix}.{k}.avi') for k in serial_nums])
except:
    print('No Frames')

Frame counts [451]


In [25]:
cam = get_camera(serial_number=sn)

In [26]:
cam.close()